In [1]:
#imports go here: adding the important ones

import os
import os.path as osp
import json

import cv2
import numpy as np
import torch
import torch.nn.functional as F

from data import load_jsrt, load_unpaired
from net import translator, utils
from scope_tools import translation_metrics
#fuck it!

In [6]:
bases = ["/home/rb080/scratch/Archives/narnia/Outputs/ab_ssim",
        "/home/rb080/scratch/Archives/narnia/Outputs/ab_ssim_attn", 
        "/home/rb080/scratch/Archives/narnia/Outputs/ab_ssim_deep",
        "/home/rb080/scratch/Archives/narnia/Outputs/unp1",
        "/home/rb080/scratch/Archives/narnia/Outputs/ab_base_repeat", 
        "/home/rb080/scratch/Archives/narnia/Outputs/ab_base_attn", 
        "/home/rb080/scratch/Archives/narnia/Outputs/ab_base_deep",
        "/home/rb080/scratch/Archives/narnia/Outputs/ab_base_attn_deep"
        ]
datasets = ["unpaired", "jsrt"]
data_paths = ["/home/rb080/scratch/Archives/narnia/Data/Unpaired", 
             "/home/rb080/scratch/Archives/narnia/Data/JSRT_dataset"]


model_ext = "saved_models/Gxy_bm.pth"
args_ext = "configs.json"





In [7]:
#Tools
class DATA():
    def __init__(self, dataset, path, bs=1, nw=0):
        self.data_root = path
        self.trans_batch = bs
        self.num_workers = nw
        self.use_attention = False
        self.dataparallel = True
        if torch.cuda.is_available(): self.device = "cuda"
        else: self.device = "cpu"
        print("Using device:", self.device)


def load_image(img_path, map_path=None, mode="torch"):
    print(img_path, map_path)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (256, 256), cv2.INTER_AREA)
    
    if mode == "cv2": return img
    
    img = np.array(img, np.float32).transpose(2, 0, 1) / 255.0
    
    if mode == "numpy": return img
    
    img = torch.tensor(img, dtype=torch.float32)
    img = img.unsqueeze(0)
    
    mask = cv2.imread(map_path, cv2.IMREAD_GRAYSCALE)
    mask = np.array(mask, np.float32) / 255.0
    mask = torch.tensor(mask, dtype=torch.float32)
    mask = mask.unsqueeze(0).unsqueeze(0)
    print(img.shape, mask.shape)
    return img, mask

def convert(T):
    T = (T[0] * 255.0).detach().cpu()
    T = np.array(T).transpose(1,2,0)
    return T

In [8]:
#data_lists
experiment_keys = ["ab_ssim", "ab_ssim_attn", "ab_ssim_deep", "ab_ssim_deep_attn",
                   "ab_base", "ab_base_attn", "ab_base_deep", "ab_base_deep_attn"]
base_list = bases
DS = 0

if torch.cuda.is_available(): device = "cuda"
else: device = "cpu"

data_root = data_paths[DS]
dataset = datasets[DS]

master_dict = {}

for i, k in enumerate(experiment_keys):
    master_dict[k] = {}
    
    args = DATA(dataset, data_root)
    if k.count("attn") != 0: args.use_attention = True
    master_dict[k]["args"] = args
    
    Gxy, _, _, _ = translator.get_model_set(args)
    Gxy.load_state_dict(torch.load(osp.join(
        bases[i], model_ext)))
    master_dict[k]["model"] = Gxy
    master_dict[k]["model"].to(device)
    master_dict[k]["model"].eval()

Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda


In [9]:
img_list = [
    "Test/testX/2893_E.png",
    "Test/testX/augmented0_0.png",
    "Test/testX/2944_E.png",
    "Test/testX/2889_E.png",
    "Test/testX/2910_E.png",
    "Test/testX/2936_E.png",
    "Test/testX/2893_E.png", 
    "Test/testX/2952_E.png",
    "Test/testX/augmented0_4.png",
    "Test/testX/augmented0_6.png"
]

for img in img_list:
    ip = osp.join(data_paths[DS], img)
    mp = osp.join(data_paths[DS], img.replace("testX", "mapX"))
    I, M = load_image(ip, mp, mode="torch")
    I.to(device)
    M.to(device)
    for exp in experiment_keys:
        Y = master_dict[exp]["model"](I, M, deep_supervision=False)
        Y = convert(Y)
        img_name = img.split("/")[-1]
        cv2.imwrite(osp.join("/home/rb080/scratch/Archives/narnia/Outputs/visuals", exp+"_"+img_name), Y)
    

/home/rb080/scratch/Archives/narnia/Data/Unpaired/Test/testX/2893_E.png /home/rb080/scratch/Archives/narnia/Data/Unpaired/Test/mapX/2893_E.png
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
/home/rb080/scratch/Archives/narnia/Data/Unpaired/Test/testX/augmented0_0.png /home/rb080/scratch/Archives/narnia/Data/Unpaired/Test/mapX/augmented0_0.png
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
/home/rb080/scratch/Archives/narnia/Data/Unpaired/Test/testX/2944_E.png /home/rb080/scratch/Archives/narnia/Data/Unpaired/Test/mapX/2944_E.png
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
/home/rb080/scratch/Archives/narnia/Data/Unpaired/Test/testX/2889_E.png /home/rb080/scratch/Archives/narnia/Data/Unpaired/Test/mapX/2889_E.png
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
/home/rb080/scratch/Archives/narnia/Data/Unpaired/Test/testX/2910_E.png /home/rb080/scratch/Archives/narnia/Data/Unpaired/Test/mapX/2910_E.png
torch.Size([1, 3, 256, 256]) torch.Size([